In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210808.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,03/08/2021 22:21:39,Black Mexicans (2018),The Challenge (2003),Museo (2018),Misterio en las Bermudas (1970),The Young and the Damned (1950),ROMANTIC COMEDIES,SPACE.,[Default to Nicolas Cage],JAPAN,"""SCORSESE ONLY MAKES GANGSTER FLICKS"""
1,03/08/2021 22:34:41,The Young and the Damned (1950),Misterio en las Bermudas (1970),Black Mexicans (2018),Museo (2018),The Challenge (2003),"""SCORSESE ONLY MAKES GANGSTER FLICKS""",JAPAN,SPACE.,ROMANTIC COMEDIES,[Default to Nicolas Cage]
2,04/08/2021 06:41:54,Museo (2018),Black Mexicans (2018),NaN,NaN,NaN,SPACE.,JAPAN,"""SCORSESE ONLY MAKES GANGSTER FLICKS""",[Default to Nicolas Cage],ROMANTIC COMEDIES
3,04/08/2021 09:46:56,Museo (2018),Black Mexicans (2018),The Challenge (2003),The Young and the Damned (1950),Misterio en las Bermudas (1970),ROMANTIC COMEDIES,SPACE.,JAPAN,"""SCORSESE ONLY MAKES GANGSTER FLICKS""",[Default to Nicolas Cage]
4,05/08/2021 10:56:02,The Challenge (2003),The Young and the Damned (1950),Museo (2018),Black Mexicans (2018),Misterio en las Bermudas (1970),JAPAN,ROMANTIC COMEDIES,[Default to Nicolas Cage],"""SCORSESE ONLY MAKES GANGSTER FLICKS""",SPACE.
5,05/08/2021 13:26:25,The Young and the Damned (1950),Misterio en las Bermudas (1970),Black Mexicans (2018),Museo (2018),The Challenge (2003),JAPAN,"""SCORSESE ONLY MAKES GANGSTER FLICKS""",SPACE.,[Default to Nicolas Cage],ROMANTIC COMEDIES


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Black Mexicans (2018)', 'The Challenge (2003)', 'Museo (2018)',
       'Misterio en las Bermudas (1970)',
       'The Young and the Damned (1950)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Black Mexicans (2018)': <Candidate('Black Mexicans (2018)')>,
 'The Challenge (2003)': <Candidate('The Challenge (2003)')>,
 'Museo (2018)': <Candidate('Museo (2018)')>,
 'Misterio en las Bermudas (1970)': <Candidate('Misterio en las Bermudas (1970)')>,
 'The Young and the Damned (1950)': <Candidate('The Young and the Damned (1950)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Black Mexicans (2018), The Challenge (2003), Museo (2018), Misterio en las Bermudas (1970), The Young and the Damned (1950))>,
 <Ballot(The Young and the Damned (1950), Misterio en las Bermudas (1970), Black Mexicans (2018), Museo (2018), The Challenge (2003))>,
 <Ballot(Museo (2018), Black Mexicans (2018))>,
 <Ballot(Museo (2018), Black Mexicans (2018), The Challenge (2003), The Young and the Damned (1950), Misterio en las Bermudas (1970))>,
 <Ballot(The Challenge (2003), The Young and the Damned (1950), Museo (2018), Black Mexicans (2018), Misterio en las Bermudas (1970))>,
 <Ballot(The Young and the Damned (1950), Misterio en las Bermudas (1970), Black Mexicans (2018), Museo (2018), The Challenge (2003))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                          Votes  Status
-------------------------------  -------  --------
The Young and the Damned (1950)        2  Hopeful
Museo (2018)                           2  Hopeful
Black Mexicans (2018)                  1  Rejected
The Challenge (2003)                   1  Rejected
Misterio en las Bermudas (1970)        0  Rejected

FINAL RESULT
Candidate                          Votes  Status
-------------------------------  -------  --------
Museo (2018)                           3  Elected
The Young and the Damned (1950)        3  Rejected
Black Mexicans (2018)                  0  Rejected
The Challenge (2003)                   0  Rejected
Misterio en las Bermudas (1970)        0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                                Votes  Status
-------------------------------------  -------  --------
JAPAN                                        2  Hopeful
ROMANTIC COMEDIES                            2  Hopeful
SPACE.                                       1  Rejected
"SCORSESE ONLY MAKES GANGSTER FLICKS"        1  Rejected
[Default to Nicolas Cage]                    0  Rejected

FINAL RESULT
Candidate                                Votes  Status
-------------------------------------  -------  --------
JAPAN                                        4  Elected
ROMANTIC COMEDIES                            2  Rejected
SPACE.                                       0  Rejected
"SCORSESE ONLY MAKES GANGSTER FLICKS"        0  Rejected
[Default to Nicolas Cage]                    0  Rejected

